# Browse & Compare Projections

Look up projections by player, system, and season. Compare what different systems predict for the same player.

In [ ]:
import pandas as pd

from fantasy_baseball_manager.db.connection import create_connection
from fantasy_baseball_manager.repos.player_repo import SqlitePlayerRepo
from fantasy_baseball_manager.repos.projection_repo import SqliteProjectionRepo
from fantasy_baseball_manager.services.projection_lookup import ProjectionLookupService

conn = create_connection("../data/fbm.db")

player_repo = SqlitePlayerRepo(conn)
projection_repo = SqliteProjectionRepo(conn)
lookup_service = ProjectionLookupService(player_repo, projection_repo)

## List Available Systems

See which projection systems are loaded for a given season.

In [ ]:
SEASON = 2025

systems = lookup_service.list_systems(SEASON)
systems_df = pd.DataFrame(
    [
        {
            "system": s.system,
            "version": s.version,
            "source_type": s.source_type,
            "batters": s.batter_count,
            "pitchers": s.pitcher_count,
        }
        for s in systems
    ]
)
systems_df

## Player Lookup

Look up a player by name and see their projections across all systems.

In [ ]:
PLAYER_NAME = "Soto"  # supports "LastName" or "LastName, FirstName"

projections = lookup_service.lookup(PLAYER_NAME, SEASON)
proj_df = pd.DataFrame(
    [
        {"player": p.player_name, "system": p.system, "version": p.version, "type": p.player_type, **p.stats}
        for p in projections
    ]
)
proj_df

## Side-by-Side Comparison

Pivot the projections so each system is a column and stats are rows.

In [ ]:
if not proj_df.empty:
    # Filter to a single player if multiple matched
    player = proj_df["player"].iloc[0]
    single = proj_df[proj_df["player"] == player].copy()

    # Melt stat columns, then pivot by system
    stat_cols = [c for c in single.columns if c not in ("player", "system", "version", "type")]
    melted = single.melt(id_vars=["system"], value_vars=stat_cols, var_name="stat")
    comparison = melted.pivot(index="stat", columns="system", values="value")
    comparison

## Ad-hoc SQL Exploration

Run queries directly against the projection table for maximum flexibility.

In [ ]:
# Example: top 10 projected HR leaders in a system
query = """
SELECT p.name_first || ' ' || p.name_last AS player,
       pr.system,
       pr.hr,
       pr.avg,
       pr.war
  FROM projection pr
  JOIN player p ON p.id = pr.player_id
 WHERE pr.season = ?
   AND pr.player_type = 'batter'
 ORDER BY pr.hr DESC
 LIMIT 10
"""
pd.read_sql(query, conn, params=[SEASON])